
# ⚡️📄 Basic RAG using Groq API + FAISS

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) setup using:

- 🧠 `Groq` API for high-speed inference with LLMs
- 📦 `FAISS` for vector database
- 🔗 `LangChain` for pipeline composition

---

## ⚙️ Prerequisites

1. **Generate your Groq Inference API Key**  
   Sign up at [https://console.groq.com](https://console.groq.com) and get your API key. Save it in your `.env` file like this:
   ```
   GROQ_API_KEY=your_key_here
   ```

2. **Install required Python package**  
   If you haven't already, install the LangChain Groq integration:

   ```bash
   pip install langchain_groq
   ```

3. **Model Selection**  
   This notebook uses `llama3-70b-8192` (aka `llama-3.3-70b-versatile`) by default.  
   You can change the model to any from Groq's available list, such as:
   - `mixtral-8x7b`
   - `gemma-7b-it`

   Update the `ChatGroq` call in the notebook accordingly.


In [1]:
# Load API keys from `.env` file for Hugging Face authentication
from dotenv import load_dotenv
import os

load_dotenv()

True

In [11]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [3]:
# Load PDF and split it into chunks
pdf_file = 'sample.pdf'
chunk_size = 1000
chunk_overlap = 200

loader = PyPDFLoader(pdf_file)
documents = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_documents(documents)

/Users/chirag/miniconda3/envs/LLMs-env/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [4]:
texts[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-06-18T14:09:48-07:00', 'moddate': '2024-06-18T14:10:14-07:00', 'trapped': '/False', 'source': 'sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Before using iPhone, review the iPhone User Guide  at  \nsupport.apple.com/guide/iphone .\nSafety and Handling\nSee “Safety, handling, and support” in the iPhone  \nUser Guide .\nExposure to Radio Frequency\nOn iPhone, go to Settings > General > Legal &  \nRegulatory > RF Exposure. Or go to apple.com/  \nlegal/rfexposure .\nBattery and Charging\nAn iPhone battery should only be repaired by a trained \ntechnician to avoid battery damage, which could cause \noverheating, fire, or injury. Batteries should be recycled \nor disposed of separately from household waste and \naccording to local environmental laws and guidelines. For \ninformation about Apple lithium-ion batteries and battery \nservice a

In [5]:
len(texts)

8

In [6]:
# 🔧 Specify the Hugging Face embedding model to use
model_name = "sentence-transformers/all-mpnet-base-v2"

# ⚙️ Model-specific arguments (e.g., run on CPU)
model_kwargs = {'device': 'cpu'}

# 🧮 Embedding behavior settings
# normalize_embeddings=False ensures raw embeddings are used (not L2-normalized)
encode_kwargs = {'normalize_embeddings': False}

# 🤗 Initialize the HuggingFaceEmbeddings class with specified settings
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [7]:
# Create FAISS vector store from the embedded chunks
vectorstore = FAISS.from_documents(
    texts,
    embedding=hf
)

In [8]:
retriever = vectorstore.as_retriever()

In [20]:
# Define the custom prompt template for generation

template = """### Instruction:
You are a customer service agent for an Apple mobile company. Your task is to answer customer queries based solely on the provided context.
You must review the context thoroughly and provide accurate, relevant responses to the customer query.
Do not make assumptions or add information beyond what is explicitly stated in the context.
Please answer in a friendly and professional manner.

If the question is not related to the context, say: "Please ask me questions related to Apple Mobiles only."

### Context:
{context}

### Question:
{query}

### Answer:"""


In [10]:
# Wrap prompt inside ChatPromptTemplate for compatibility with chat models
prompt = ChatPromptTemplate.from_messages(
    [
	("system", "You are a helpful assistant."),
	("human", template),
    ]
)

print(prompt)

input_variables=['context', 'query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\nYou are a customer service agent for a apple mobile company. \nYou have been given the following information about the customer query and the context.\nCustomer Query: {query}\nContext: {context}\n\n### Answer: \nThe answer should be based on the context provided.\nYour task is to answer the customer query based on the context provided. If the question is not related to the context, please say "I don\'t know or Do Not Answer it just say please ask me question related to Apple Mobiles only".\nDo not make up any information or provide any personal opinions or experiences.\nPlease answer in a friendly a

In [ ]:
# 🧠 Load LLM using Groq API (make sure GROQ_API_KEY is set)
# Default I am using `llama3-70b-8192 (aka llama-3.3-70b-versatile)` model

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",  # other options: "llama2-70b-4096", "gemma-7b-it"
    temperature=0.2,
)

In [25]:
# Combine content of all retrieved documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
# Example 1: Ask a domain-specific question
response = rag_chain.invoke('Is there a warranty on the phone?')
print('📦 Answer:', response)

📦 Answer: Thank you for reaching out to us about your query on the warranty of your Apple phone. I'd be happy to help. According to our warranty policy, Apple provides a one-year limited warranty on the hardware product and accessories against defects in materials and workmanship from the date of original retail purchase. However, please note that this warranty does not cover normal wear and tear, nor damage caused by accident or abuse. 

To obtain service, you can call Apple or visit an Apple Store or an Apple Authorized Service Provider. The available service options may vary depending on the country in which service is requested and may be restricted to the original country of sale. You can find more detailed information on obtaining service at apple.com/legal/warranty and support.apple.com.

If you have any further questions or would like to know more about the warranty, please don't hesitate to ask.


In [28]:
# Example 2: Ask a general knowledge question
response = rag_chain.invoke('What is GTA 6?')
print('🎮 Answer:', response)

🎮 Answer: I don't know. Please ask me a question related to Apple Mobiles only. The information provided is about iPhone safety, handling, and regulatory information, but it does not mention GTA 6. I'd be happy to help with any questions you have about Apple Mobiles.


<!-- Font Awesome CDN (Add in <head> if not already included) -->
<link
  rel="stylesheet" 
  href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
/>

<!-- Social Footer Section -->
<div style="
  background-color:rgb(199, 195, 195);
  padding: 40px 30px;
  border-radius: 20px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.08);
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  font-size: 18px;
  max-width: 900px;
  margin: 60px auto 30px;
  text-align: center;
  color: #444;
">
<!-- End of Notebook Note -->
  <h2 style="margin-bottom: 10px;">📘 End of Notebook</h2>
  <p style="color: #666; font-size: 14px;">
    Thank you for exploring! Feel free to connect via the links below.
  </p>

  <!-- Social Icons -->
<div style="
  display: flex;
  gap: 25px;
  align-items: center;
  flex-wrap: wrap;
  justify-content: center;
  margin-bottom: 25px;
">
  <!-- LinkedIn -->
  <a href="https://www.linkedin.com/in/ChiragB254" target="_blank" style="text-decoration: none; color: #0077b5;">
    <i class="fab fa-linkedin fa-lg"></i> LinkedIn
  </a>

  <!-- GitHub -->
  <a href="https://github.com/ChiragB254" target="_blank" style="text-decoration: none; color: #333;">
    <i class="fab fa-github fa-lg"></i> GitHub
  </a>

  <!-- Instagram -->
  <a href="https://www.instagram.com/data.scientist_chirag" target="_blank" style="text-decoration: none; color: #E1306C;">
    <i class="fab fa-instagram fa-lg"></i> Instagram
  </a>

  <!-- Email -->
  <a href="mailto:devchirag27@gmail.com" style="text-decoration: none; color: #D44638;">
    <i class="fas fa-envelope fa-lg"></i> Email
  </a>

  <!-- X (Twitter) -->
  <a href="https://x.com/ChiragB254" target="_blank" style="text-decoration: none; color: #000;">
    <i class="fab fa-x-twitter fa-lg"></i> X.com
  </a>
  </div>

  <p style="font-size: 13px; color: black; font-style: italic; margin-top: 8px;">
    <strong>Made with ❤️ by Chirag Bansal</strong>
  </p>
</div>

